# **Detección de anomalías: Enfermedad de la tiroides**

En la siguente notebook se aplicarán y se compararán tres métodos de detección de anomalías a la dataset [¨Thyroid Disease Unsupervised Anomaly Detection¨](https://www.kaggle.com/zhonglifr/thyroid-disease-unsupervised-anomaly-detection). Los métodos utilizados son **Isolation Forest**, **One-Class SVM** y **Local Outlier Factor.**

# Imports

In [95]:
# Importamos la librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import RocCurveDisplay, auc, roc_curve, precision_recall_fscore_support, accuracy_score, classification_report

In [96]:
# Se lee la dataset
df = pd.read_csv("../annthyroid_unsupervised_anomaly_detection.csv", delimiter=';')
df.head()

# Preprocesado

In [97]:
# Se eliminan las columnas que tienen valores NaN
df = df.drop(["Unnamed: 22", "Unnamed: 23"], axis=1)
df.head()

In [98]:
# Se categorizan los labels en 0 y 1
df["Outlier_label "] = df["Outlier_label "].astype("category").cat.codes
df.head()

In [99]:
df.describe()

In [100]:
df.info()

Tenemos 6916 datos para comenzar a trabajar.

## Estandarización de datos

In [101]:
tmp_labels = df['Outlier_label ']
df = df.drop(['Outlier_label '], axis=1)
df = (df-df.mean())/df.std()
df['Outlier_label '] = tmp_labels

## Correlación

In [102]:
corr = df.corr()
sns.heatmap(corr)

In [103]:
print(corr['Outlier_label '])

Podemos observar que hay muy poca correlación entre las columnas, siendo TSH la que tiene más relación en cuanto a si se tiene enfermedad de la tiroides o no.

## Separando datos y labels

In [104]:
X = df.drop(['Outlier_label '], axis=1)
Y = df['Outlier_label ']

In [105]:
n_outliers = len(df[df['Outlier_label ']==1])
outlier_fraction = len(df[df['Outlier_label ']==1])/float(len(df[df['Outlier_label ']==0]))

In [106]:
n_outliers

De los 6916 datos, tenemos 250 outliers, lo cuales son los datos de personas que padecen enfermedad de la tiroides.

# Método: Isolation Forest

In [107]:
params = {'n_estimators': list(range(5, 15)), 'max_features': list(range(2, 10))}
# Se obtienen los mejores parámetros con GridSearchCV
clf1 = GridSearchCV(IsolationForest(n_jobs=2, random_state=5, contamination=outlier_fraction), params, scoring='homogeneity_score', cv=5)
clf1.fit(X, Y)
scores_prediction = clf1.decision_function(X)
y1_pred = clf1.predict(X)
y1_pred[y1_pred == 1] = 0
y1_pred[y1_pred == -1] = 1
n_errors = (y1_pred != Y).sum()

In [108]:
# Se imprimen los parámetros utilizados
print(clf1.best_params_)

In [109]:
print("{}: {}".format("Puntos anómalos con Isolation Forest",n_errors))
print("Precisión:",accuracy_score(Y,y1_pred))
print(classification_report(Y,y1_pred))

In [110]:
fpr1, tpr1, _ = roc_curve(Y, y1_pred)
roc1_auc = auc(fpr1, tpr1)
rcd1 = RocCurveDisplay(fpr=fpr1, tpr=tpr1, roc_auc=roc1_auc)
rcd1.plot()

Utilizando Isolation Forest conseguimos una precisión de aproximadamente el 95.28% con un valor de AUC calculado con la curva ROC de 0.67, lo cual lo denomina como regular. Predijo 326 anomalías teniendo algunos falsos positivos.

# Método: One-Class SVM

In [111]:
clf2 = OneClassSVM(kernel="rbf", nu=outlier_fraction)
y2_pred = clf2.fit_predict(X)
y2_pred[y2_pred == 1] = 0
y2_pred[y2_pred == -1] = 1
m_errors = (y2_pred != Y).sum()
print("{}: {}".format("Puntos anómalos con One-Class SVM",m_errors))
print("Precisión:", accuracy_score(Y,y2_pred))
print(classification_report(Y,y2_pred))

Utilizando One-Class SVM conseguimos una precisión de aproximadamente el 93.34%. Predijo 460 anomalías teniendo varios falsos positivos.

# Método: Local Outlier Factor

In [112]:
clf3 = LocalOutlierFactor(n_neighbors=60, algorithm='auto', contamination=outlier_fraction)
y3_pred = clf3.fit_predict(X)
y3_pred[y3_pred == 1] = 0
y3_pred[y3_pred == -1] = 1
k_errors = (y3_pred != Y).sum()
print("{}: {}".format("Puntos anómalos con Local Outlier Factor",k_errors))
print("Precisión:", accuracy_score(Y,y3_pred))
print(classification_report(Y,y3_pred))

Utilizando Local Outlier Factor conseguimos una precisión de aproximadamente el 94.62%. Predijo 372 anomalías teniendo algunos falsos positivos.

# Conclusión

De los tres métodos para detección de anomalías utilizados Isolation Forest tuvo una mejor precisión detectando 326 anomalías con 95.28% de precisión aproximadamente y con solo 76 falsos positivos. En segundo lugar está Local Outlier Factor detectando 372 anomalías con 94.62% de precisión aproximadamente y 122 falsos positivos. En tercer y último lugar queda One-Class SVM detectando 460 anomalías con 93.34% de precisión aproximadamente y 210 falsos positivos lo cual ya es bastante comparado con los otros dos métodos.